# CA Gubernatorial Race

This is a new twitter scrape notebook for the California Gubernatorial race.

The current list of announced and possible candidates that the project will be tracking are the ones listed on _La Times_ article [_California's next governor: Who's running, who's on the fence?_](http://www.latimes.com/politics/la-pol-ca-california-governor-list-2018-htmlstory.html) 

This is a list of the candidates listed in the article as of September 6th, 2017:

**Announced**
* [Gavin Newsom - D](https://twitter.com/GavinNewsom) [02/10/2015](http://www.latimes.com/local/politics/la-me-pol-gavin-newsom-20150212-story.html)
* [John Chiang - D](https://twitter.com/JohnChiangCA) [05/17/2016](http://www.latimes.com/politics/la-pol-sac-essential-poli-john-chiang-jumps-into-californias-2018-governor-1463506797-htmlstory.html)
* [Antonio Villaraigosa - D](https://twitter.com/antonio4ca) [11/10/2016](http://www.dailynews.com/2016/11/10/former-la-mayor-antonio-villaraigosa-launches-bid-for-california-governor/)
* [Delaine Eastin - D](https://twitter.com/DelaineEastin) [11/01/2016](https://ballotpedia.org/Delaine_Eastin)
* [John Cox - R](https://twitter.com/TheRealJohnHCox) [03/07/2017](https://en.wikipedia.org/wiki/John_H._Cox#2018_California_gubernatorial_election)
* [Travis Cox - R](https://twitter.com/JoinTravisAllen) [06/22/2017](https://ballotpedia.org/Travis_Allen)
* [Zoltan Istvan - L](https://twitter.com/zoltan_istvan) [02/12/2017](http://www.newsweek.com/zoltan-istvan-california-governor-libertarian-555088)

**Speculated**
* [Kevin Faulconer](https://twitter.com/Kevin_Faulconer)
* [Eric Garcetti](https://twitter.com/ericgarcetti)
* [Tom Steyer](https://twitter.com/TomSteyer)
* [Ashley Sweargin](https://twitter.com/ashleycvcf)
* [Steve Westly](https://twitter.com/SteveWestly)